In [5]:
"""
generate_rti_html_responses.ipynb

This script reads a CSV or Excel file containing RTI numbers and user queries,
sends the queries to an LLM-powered chatbot (e.g., RTI Mitra) via an OpenAI-compatible API,
and generates individual HTML files and one combined HTML document with the drafted RTI responses.

Features:
- Handles missing and duplicate RTI numbers
- Supports Markdown rendering of responses
- Provides both individual and consolidated HTML output
- Logs processing time for each RTI

Requirements:
- Python 3.7+
- pandas, requests, markdown, python-dotenv

Usage:
    Place your input file (CSV or XLSX) at the specified INPUT_PATH.
    Configure API_URL and MODEL_ID as needed.
    Run the script to generate HTML files in the OUTPUT_DIR.

Author: OnlineRTI / RTI Mitra Team
"""

# === IMPORTS ===
import os, re, html, json
import pandas as pd
import requests
import markdown
import time
from dotenv import load_dotenv

# === ENV SETUP ===
# Load environment variables from a .env file (e.g., API key for AnythingLLM)
load_dotenv()

# === CONFIGURATION ===
INPUT_PATH = "test_input.csv"        # Input CSV/Excel file containing RTI numbers and user queries
OUTPUT_DIR = "rti_html_outputs"           # Directory where individual HTML outputs will be saved
COMBINED_HTML = "RTI_Mitra_All.html"      # File to save all RTI drafts combined in one HTML
REQUIRED_COLUMNS = ["rtinumber", "user query"]  # Columns required from the input file

API_URL = "http://localhost:3001/api/v1/openai/chat/completions"  # API endpoint for LLM
MODEL_ID = "rti-mitra"                        # Name of the deployed model or workspace alias
api_key = os.getenv("ANYTHINGLLM_API_KEY")   # Load API key from environment (if required)

# HTTP headers for API request
HEADERS = {"Content-Type": "application/json"}
if api_key:
    HEADERS["Authorization"] = f"Bearer {api_key}"

SYSTEM_MESSAGE = "Please draft RTI as per user query"  # System prompt for the model

# Policies for handling invalid or duplicate RTI numbers
ON_MISSING_RTI = "skip"       # Options: "skip" (ignore row), "autofill" (generate dummy RTI number)
ON_DUPLICATE_RTI = "suffix"   # Options: "suffix" (add __2, __3...), "error" (stop execution)

# === HELPER FUNCTIONS ===

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize column names by lowercasing and trimming spaces."""
    df = df.copy()
    df.columns = [c.strip().lower() for c in df.columns]
    return df

def coerce_str_strip(series: pd.Series) -> pd.Series:
    """Convert values to strings and remove leading/trailing spaces."""
    return series.astype(str).str.strip()

def safe_filename(s: str, max_len: int = 120) -> str:
    """Sanitize a string to use as a filename."""
    s = re.sub(r"[^A-Za-z0-9._-]+", "_", s.strip())
    return s[:max_len]

def handle_missing_rti(df: pd.DataFrame) -> pd.DataFrame:
    """Process rows with missing RTINumber as per configured policy."""
    mask_blank = df["rtinumber"].isin(["", "nan", "none", "null"])
    missing_count = int(mask_blank.sum())
    if not missing_count:
        return df

    if ON_MISSING_RTI == "skip":
        print(f"⚠️  Skipping {missing_count} row(s) with empty RTINumber.")
        return df.loc[~mask_blank].copy()

    if ON_MISSING_RTI == "autofill":
        print(f"ℹ️  Auto-filling {missing_count} missing RTINumber value(s).")
        counter = 1
        new_vals = []
        for is_blank in mask_blank.tolist():
            if is_blank:
                new_vals.append(f"AUTO_{counter:05d}")
                counter += 1
            else:
                new_vals.append(None)
        df2 = df.copy()
        df2.loc[mask_blank, "rtinumber"] = [x for x in new_vals if x is not None][:missing_count]
        return df2

    raise ValueError(f"Unknown ON_MISSING_RTI='{ON_MISSING_RTI}'")

def handle_duplicate_rti(df: pd.DataFrame) -> pd.DataFrame:
    """Make RTINumber values unique if duplicates exist."""
    dup_mask = df["rtinumber"].duplicated(keep=False)
    if not dup_mask.any():
        return df

    if ON_DUPLICATE_RTI == "error":
        dups = df.loc[dup_mask, "rtinumber"].tolist()
        raise ValueError(f"Duplicate RTINumber(s) found: {dups[:10]}{'...' if len(dups)>10 else ''}")

    if ON_DUPLICATE_RTI == "suffix":
        print("ℹ️  Duplicates detected. Appending numeric suffixes to make filenames unique.")
        counts = {}
        new_vals = []
        for val in df["rtinumber"]:
            c = counts.get(val, 0)
            counts[val] = c + 1
            new_vals.append(f"{val}__{c+1}" if c > 0 else val)
        df2 = df.copy()
        df2["rtinumber"] = new_vals
        return df2

    raise ValueError(f"Unknown ON_DUPLICATE_RTI='{ON_DUPLICATE_RTI}'")

def call_chat_api(user_query: str) -> str:
    """Send the user query to the LLM API and return the generated response."""
    payload = {
        "model": MODEL_ID,
        "messages": [
            {"role": "user", "content": user_query}
        ]
    }
    resp = requests.post(API_URL, headers=HEADERS, data=json.dumps(payload), timeout=120)
    resp.raise_for_status()
    data = resp.json()
    try:
        return data["choices"][0]["message"]["content"]
    except Exception:
        return json.dumps(data, ensure_ascii=False)  # fallback for debugging

def format_with_linebreaks(text: str) -> str:
    """Escape HTML and convert newlines to <br> for browser display."""
    if not text:
        return ''
    text = text.replace("\\n", "\n")
    return html.escape(text).replace("\n", "<br>")

def render_markdown_nl2br(text: str) -> str:
    """Convert markdown to HTML with newline support."""
    return markdown.markdown(text or "", extensions=["nl2br"])

def save_html(filename: str, rti_number: str, user_query: str, response: str, is_error=False):
    """Save a single RTI and its response to an HTML file."""
    rti_number_html = html.escape(rti_number)
    user_query_html = format_with_linebreaks(user_query)
    response_html = render_markdown_nl2br(response)
    if is_error:
        response_html = f'<div class="error" style="color:red; font-weight:bold;">{response_html}</div>'

    html_doc = f"""
    <html>
      <head>
        <meta charset="utf-8" />
        <title>{html.escape(rti_number)} - RTI Mitra</title>
        <style>
          body {{ font-family: Arial, sans-serif; margin: 20px; }}
          h1, h2, h3 {{ color: #333; }}
        </style>
      </head>
      <body>
        <div style="border:1px solid #ccc; padding:1em; margin-bottom:2em; background-color:#f9f9f9;">
          <h2 style="color:blue;">RTI Number:</h2>
          {rti_number_html}
          <h3 style="color:blue;">User Query</h3>
          {user_query_html}
          <h3 style="color:blue;">Drafted RTI</h3>
          {response_html}
        </div>
        <hr style="border:1px solid #aaa; margin:2em 0;">
      </body>
    </html>
    """
    with open(filename, "w", encoding="utf-8") as f:
        f.write(html_doc)

# === MAIN LOGIC ===

# Step 1: Read input data
df = pd.read_excel(INPUT_PATH) if INPUT_PATH.lower().endswith(".xlsx") else pd.read_csv(INPUT_PATH)

# Step 2: Standardize column names and map to required ones
df = normalize_columns(df)
col_map = {}
for c in df.columns:
    c_clean = c.replace(" ", "")
    if c_clean == "rtinumber":
        col_map[c] = "rtinumber"
    elif c_clean in ("userquery", "user_message", "message", "query"):
        col_map[c] = "user query"
df = df.rename(columns=col_map)

# Step 3: Validate required columns
missing_req = [c for c in REQUIRED_COLUMNS if c not in df.columns]
if missing_req:
    raise ValueError(f"Missing required column(s): {missing_req}. Available: {list(df.columns)}")

# Step 4: Clean string data
df["rtinumber"] = coerce_str_strip(df["rtinumber"]).str.lower().replace({"nan": "", "none": "", "null": ""})
df["user query"] = coerce_str_strip(df["user query"])

# Step 5: Handle missing or duplicate RTI numbers
df = handle_missing_rti(df)
df = handle_duplicate_rti(df)
df = df[df["rtinumber"] != ""].copy()  # Final cleanup

# Step 6: Create output folder
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Step 7: Loop through queries, call API, and generate HTMLs
all_blocks = []
durations = []

for _, row in df.iterrows():
    rti_number = str(row["rtinumber"]).strip()
    user_query = str(row["user query"]).strip()
    rti_for_filename = safe_filename(rti_number)
    out_file = os.path.join(OUTPUT_DIR, f"{rti_for_filename}_RTIMitra.html")

    print(f"Processing RTI {rti_number} -> {os.path.basename(out_file)}")
    is_error = False
    start = time.perf_counter()

    try:
        response = call_chat_api(user_query)
    except Exception as e:
        response = f"ERROR: {e}"
        is_error = True

    elapsed = time.perf_counter() - start
    durations.append(elapsed)
    print(f"  -> Processed in {elapsed:.3f} seconds")

    save_html(out_file, rti_number, user_query, response, is_error=is_error)

    block = f"""
    <div style="border:1px solid #ccc; padding:1em; margin-bottom:2em; background-color:#f9f9f9;">
      <h2 style="color:blue;">RTI Number:</h2>
      {html.escape(rti_number)}
      <h3 style="color:blue;">User Query</h3>
      {format_with_linebreaks(user_query)}
      <h3 style="color:blue;">Drafted RTI</h3>
      {'<div class="error" style="color:red; font-weight:bold;">' + render_markdown_nl2br(response) + '</div>' if is_error
        else render_markdown_nl2br(response)}
    </div>
    <hr style="border:1px solid #aaa; margin:2em 0;">
    """
    all_blocks.append(block)

# Step 8: Summary + Combine all HTML outputs
avg_time = sum(durations) / len(durations) if durations else 0.0
summary_html = f"""
<div style="margin-top:2em; padding:1em; border:1px dashed #bbb; background:#fafafa;">
  <strong>Processed {len(durations)} RTI(s)</strong><br>
  Average time per RTI: {avg_time:.3f} s
</div>
"""

combined_html = f"""
<html>
  <head>
    <meta charset="utf-8" />
    <title>RTI Mitra - All Drafts</title>
    <style>
      .error {{ color:#b91c1c; font-weight:bold; }}
      body {{ font-family: Arial, sans-serif; margin: 20px; }}
      h1 {{ color:#333; }}
    </style>
  </head>
  <body>
    <h1>RTI Mitra - All Drafts</h1>
    {''.join(all_blocks)}
    {summary_html}
  </body>
</html>
"""
with open(COMBINED_HTML, "w", encoding="utf-8") as f:
    f.write(combined_html)

# Final console log
print(f"✅ Done. Individual files in '{OUTPUT_DIR}', combined file: {COMBINED_HTML}")
print(f"ℹ️ Average time per RTI: {avg_time:.3f} s over {len(durations)} item(s).")

Processing RTI 1 -> 1_RTIMitra.html
  -> Processed in 6.422 seconds
Processing RTI 2 -> 2_RTIMitra.html
  -> Processed in 5.181 seconds
✅ Done. Individual files in 'rti_html_outputs', combined file: RTI_Mitra_All.html
ℹ️ Average time per RTI: 5.802 s over 2 item(s).


In [ ]:
"""
generate_rti_html_responses.ipynb

This script reads a CSV or Excel file containing RTI numbers and user queries,
sends the queries to an LLM-powered chatbot (e.g., RTI Mitra) via an OpenAI-compatible API,
and generates individual HTML files and one combined HTML document with the drafted RTI responses.

Features:
- Handles missing and duplicate RTI numbers
- Supports Markdown rendering of responses
- Provides both individual and consolidated HTML output
- Logs processing time for each RTI

Requirements:
- Python 3.7+
- pandas, requests, markdown, python-dotenv

Usage:
    Place your input file (CSV or XLSX) at the specified INPUT_PATH.
    Configure API_URL and MODEL_ID as needed.
    Run the script to generate HTML files in the OUTPUT_DIR.

Author: OnlineRTI / RTI Mitra Team
"""

# === IMPORTS ===
import os, re, html, json
import pandas as pd
import requests
import markdown
import time
from dotenv import load_dotenv

# === ENV SETUP ===
# Load environment variables from a .env file (e.g., API key for AnythingLLM)
load_dotenv()

# === CONFIGURATION ===
INPUT_PATH = "test_input.csv"        # Input CSV/Excel file containing RTI numbers and user queries
OUTPUT_DIR = "rti_html_outputs"           # Directory where individual HTML outputs will be saved
COMBINED_HTML = "RTI_Mitra_All.html"      # File to save all RTI drafts combined in one HTML
REQUIRED_COLUMNS = ["rtinumber", "user query"]  # Columns required from the input file

API_URL = "http://localhost:3001/api/v1/openai/chat/completions"  # API endpoint for LLM
MODEL_ID = "rti-mitra"                        # Name of the deployed model or workspace alias
api_key = os.getenv("ANYTHINGLLM_API_KEY")   # Load API key from environment (if required)

# HTTP headers for API request
HEADERS = {"Content-Type": "application/json"}
if api_key:
    HEADERS["Authorization"] = f"Bearer {api_key}"

SYSTEM_MESSAGE = "Please draft RTI as per user query"  # System prompt for the model

# Policies for handling invalid or duplicate RTI numbers
ON_MISSING_RTI = "skip"       # Options: "skip" (ignore row), "autofill" (generate dummy RTI number)
ON_DUPLICATE_RTI = "suffix"   # Options: "suffix" (add __2, __3...), "error" (stop execution)

# === HELPER FUNCTIONS ===

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize column names by lowercasing and trimming spaces."""
    df = df.copy()
    df.columns = [c.strip().lower() for c in df.columns]
    return df

def coerce_str_strip(series: pd.Series) -> pd.Series:
    """Convert values to strings and remove leading/trailing spaces."""
    return series.astype(str).str.strip()

def safe_filename(s: str, max_len: int = 120) -> str:
    """Sanitize a string to use as a filename."""
    s = re.sub(r"[^A-Za-z0-9._-]+", "_", s.strip())
    return s[:max_len]

def handle_missing_rti(df: pd.DataFrame) -> pd.DataFrame:
    """Process rows with missing RTINumber as per configured policy."""
    mask_blank = df["rtinumber"].isin(["", "nan", "none", "null"])
    missing_count = int(mask_blank.sum())
    if not missing_count:
        return df

    if ON_MISSING_RTI == "skip":
        print(f"⚠️  Skipping {missing_count} row(s) with empty RTINumber.")
        return df.loc[~mask_blank].copy()

    if ON_MISSING_RTI == "autofill":
        print(f"ℹ️  Auto-filling {missing_count} missing RTINumber value(s).")
        counter = 1
        new_vals = []
        for is_blank in mask_blank.tolist():
            if is_blank:
                new_vals.append(f"AUTO_{counter:05d}")
                counter += 1
            else:
                new_vals.append(None)
        df2 = df.copy()
        df2.loc[mask_blank, "rtinumber"] = [x for x in new_vals if x is not None][:missing_count]
        return df2

    raise ValueError(f"Unknown ON_MISSING_RTI='{ON_MISSING_RTI}'")

def handle_duplicate_rti(df: pd.DataFrame) -> pd.DataFrame:
    """Make RTINumber values unique if duplicates exist."""
    dup_mask = df["rtinumber"].duplicated(keep=False)
    if not dup_mask.any():
        return df

    if ON_DUPLICATE_RTI == "error":
        dups = df.loc[dup_mask, "rtinumber"].tolist()
        raise ValueError(f"Duplicate RTINumber(s) found: {dups[:10]}{'...' if len(dups)>10 else ''}")

    if ON_DUPLICATE_RTI == "suffix":
        print("ℹ️  Duplicates detected. Appending numeric suffixes to make filenames unique.")
        counts = {}
        new_vals = []
        for val in df["rtinumber"]:
            c = counts.get(val, 0)
            counts[val] = c + 1
            new_vals.append(f"{val}__{c+1}" if c > 0 else val)
        df2 = df.copy()
        df2["rtinumber"] = new_vals
        return df2

    raise ValueError(f"Unknown ON_DUPLICATE_RTI='{ON_DUPLICATE_RTI}'")

def call_chat_api(user_query: str) -> str:
    """Send the user query to the LLM API and return the generated response."""
    payload = {
        "model": MODEL_ID,
        "messages": [
            {"role": "user", "content": user_query}
        ]
    }
    resp = requests.post(API_URL, headers=HEADERS, data=json.dumps(payload), timeout=120)
    resp.raise_for_status()
    data = resp.json()
    try:
        return data["choices"][0]["message"]["content"]
    except Exception:
        return json.dumps(data, ensure_ascii=False)  # fallback for debugging

def format_with_linebreaks(text: str) -> str:
    """Escape HTML and convert newlines to <br> for browser display."""
    if not text:
        return ''
    text = text.replace("\\n", "\n")
    return html.escape(text).replace("\n", "<br>")

def render_markdown_nl2br(text: str) -> str:
    """Convert markdown to HTML with newline support."""
    return markdown.markdown(text or "", extensions=["nl2br"])

def save_html(filename: str, rti_number: str, user_query: str, response: str, is_error=False):
    """Save a single RTI and its response to an HTML file."""
    rti_number_html = html.escape(rti_number)
    user_query_html = format_with_linebreaks(user_query)
    response_html = render_markdown_nl2br(response)
    if is_error:
        response_html = f'<div class="error" style="color:red; font-weight:bold;">{response_html}</div>'

    html_doc = f"""
    <html>
      <head>
        <meta charset="utf-8" />
        <title>{html.escape(rti_number)} - RTI Mitra</title>
        <style>
          body {{ font-family: Arial, sans-serif; margin: 20px; }}
          h1, h2, h3 {{ color: #333; }}
        </style>
      </head>
      <body>
        <div style="border:1px solid #ccc; padding:1em; margin-bottom:2em; background-color:#f9f9f9;">
          <h2 style="color:blue;">RTI Number:</h2>
          {rti_number_html}
          <h3 style="color:blue;">User Query</h3>
          {user_query_html}
          <h3 style="color:blue;">Drafted RTI</h3>
          {response_html}
        </div>
        <hr style="border:1px solid #aaa; margin:2em 0;">
      </body>
    </html>
    """
    with open(filename, "w", encoding="utf-8") as f:
        f.write(html_doc)

# === MAIN LOGIC ===

# Step 1: Read input data
df = pd.read_excel(INPUT_PATH) if INPUT_PATH.lower().endswith(".xlsx") else pd.read_csv(INPUT_PATH)

# Step 2: Standardize column names and map to required ones
df = normalize_columns(df)
col_map = {}
for c in df.columns:
    c_clean = c.replace(" ", "")
    if c_clean == "rtinumber":
        col_map[c] = "rtinumber"
    elif c_clean in ("userquery", "user_message", "message", "query"):
        col_map[c] = "user query"
df = df.rename(columns=col_map)

# Step 3: Validate required columns
missing_req = [c for c in REQUIRED_COLUMNS if c not in df.columns]
if missing_req:
    raise ValueError(f"Missing required column(s): {missing_req}. Available: {list(df.columns)}")

# Step 4: Clean string data
df["rtinumber"] = coerce_str_strip(df["rtinumber"]).str.lower().replace({"nan": "", "none": "", "null": ""})
df["user query"] = coerce_str_strip(df["user query"])

# Step 5: Handle missing or duplicate RTI numbers
df = handle_missing_rti(df)
df = handle_duplicate_rti(df)
df = df[df["rtinumber"] != ""].copy()  # Final cleanup

# Step 6: Create output folder
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Step 7: Loop through queries, call API, and generate HTMLs
all_blocks = []
durations = []

for _, row in df.iterrows():
    rti_number = str(row["rtinumber"]).strip()
    user_query = str(row["user query"]).strip()
    rti_for_filename = safe_filename(rti_number)
    out_file = os.path.join(OUTPUT_DIR, f"{rti_for_filename}_RTIMitra.html")

    print(f"Processing RTI {rti_number} -> {os.path.basename(out_file)}")
    is_error = False
    start = time.perf_counter()

    try:
        response = call_chat_api(user_query)
    except Exception as e:
        response = f"ERROR: {e}"
        is_error = True

    elapsed = time.perf_counter() - start
    durations.append(elapsed)
    print(f"  -> Processed in {elapsed:.3f} seconds")

    save_html(out_file, rti_number, user_query, response, is_error=is_error)

    block = f"""
    <div style="border:1px solid #ccc; padding:1em; margin-bottom:2em; background-color:#f9f9f9;">
      <h2 style="color:blue;">RTI Number:</h2>
      {html.escape(rti_number)}
      <h3 style="color:blue;">User Query</h3>
      {format_with_linebreaks(user_query)}
      <h3 style="color:blue;">Drafted RTI</h3>
      {'<div class="error" style="color:red; font-weight:bold;">' + render_markdown_nl2br(response) + '</div>' if is_error
        else render_markdown_nl2br(response)}
    </div>
    <hr style="border:1px solid #aaa; margin:2em 0;">
    """
    all_blocks.append(block)

# Step 8: Summary + Combine all HTML outputs
avg_time = sum(durations) / len(durations) if durations else 0.0
summary_html = f"""
<div style="margin-top:2em; padding:1em; border:1px dashed #bbb; background:#fafafa;">
  <strong>Processed {len(durations)} RTI(s)</strong><br>
  Average time per RTI: {avg_time:.3f} s
</div>
"""

combined_html = f"""
<html>
  <head>
    <meta charset="utf-8" />
    <title>RTI Mitra - All Drafts</title>
    <style>
      .error {{ color:#b91c1c; font-weight:bold; }}
      body {{ font-family: Arial, sans-serif; margin: 20px; }}
      h1 {{ color:#333; }}
    </style>
  </head>
  <body>
    <h1>RTI Mitra - All Drafts</h1>
    {''.join(all_blocks)}
    {summary_html}
  </body>
</html>
"""
with open(COMBINED_HTML, "w", encoding="utf-8") as f:
    f.write(combined_html)

# Final console log
print(f"✅ Done. Individual files in '{OUTPUT_DIR}', combined file: {COMBINED_HTML}")
print(f"ℹ️ Average time per RTI: {avg_time:.3f} s over {len(durations)} item(s).")